<div style="background-color:#000;"><img src="pqn.png"></img></div>

### Set up our environment

We import necessary libraries and set up our environment for the pairs trading strategy implementation.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import (
    FunctionAgent,
    AgentWorkflow,
    AgentOutput,
    ToolCall,
    ToolCallResult,
)
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex

In [ ]:
load_dotenv()

We import libraries for file handling, asynchronous programming, and environment variable management. We also import components from the llama_index library for natural language processing and workflow management. The load_dotenv() function loads environment variables from a .env file, which is useful for storing sensitive information like API keys.

### Initialize our LLM

We set up our Language Model (LLM) using OpenAI's GPT-4.

In [ ]:
llm = OpenAI(model="gpt-4o")

We create an instance of the OpenAI language model using the GPT-4 variant. This model will be used throughout our workflow to process natural language and generate responses. The API key for OpenAI should be stored in the .env file that was loaded earlier.

### Define our workflow tools

We define the tools our agents will use in the workflow.

In [ ]:
async def read_pdf_tool(ctx: Context) -> str:
    """
    Reads the provided PDF file and extracts a detailed description of the pairs trading strategy.
    """
    documents = SimpleDirectoryReader(input_files=["pairs.pdf"]).load_data()
    index = GPTVectorStoreIndex(documents)
    query_engine = index.as_query_engine()
    query = (
        "Extract a detailed description of the pairs trading strategy implementation from this PDF. "
        "Ensure the description is detailed enough to reproduce the strategy in code."
    )
    response = query_engine.query(query)
    return str(response)

In [ ]:
async def build_plan_tool(ctx: Context, plan: str) -> str:
    """
    Uses the provided strategy description to generate a detailed implementation plan in Python.
    Updates the workflow state with the plan.
    """
    current_state = await ctx.get("state")
    current_state["implementation_plan"] = plan
    await ctx.set("state", current_state)
    return "Implementation plan recorded."

In [ ]:
async def write_code_tool(ctx: Context, plan: str) -> str:
    """
    Uses the provided implementation plan to generate complete, executable Python code.
    Updates the workflow state with the generated code.
    """
    current_state = await ctx.get("state")
    current_state["python_code"] = plan
    await ctx.set("state", current_state)
    return "Python code recorded."

We define three asynchronous functions that serve as tools in our workflow. The read_pdf_tool extracts information from a PDF file, the build_plan_tool generates an implementation plan, and the write_code_tool generates Python code. These tools interact with the workflow context to update the state as they progress.

### Create our agents

We set up the agents that will use our tools to process the pairs trading strategy.

In [ ]:
pdf_reader_agent = FunctionAgent(
    name="PDFReaderAgent",
    description=(
        "Reads a PDF file containing a pairs trading strategy and extracts a detailed description "
        "of the strategy implementation."
    ),
    system_prompt=(
        "You are the PDFReaderAgent that can read PDFs containing implementation details of pairs trading strategies "
        "and describe the strategy in detail. Once you read the PDF and describe the implementation details and are "
        "satisfied, you should hand off control to the PlanBuilderAgent to develop an implementation plan. "
        "You should have a detailed description of the strategy before handing off control to the PlanBuilderAgent."
    ),
    llm=llm,
    tools=[read_pdf_tool],
    can_handoff_to=["PlanBuilderAgent"],
)

In [ ]:
plan_builder_agent = FunctionAgent(
    name="PlanBuilderAgent",
    description=(
        "Takes the detailed strategy description and builds a detailed plan to implement the strategy in Python."
    ),

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.